In [31]:
import pandas as pd

In [32]:
#path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'
path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'

In [33]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [34]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [35]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2024-05-16 00:00:00,4.127295,221.918352,NaN
1,1,2024-05-31 00:00:00,45.972261,2338.489409,2876.77
2,1,2024-06-05 00:00:00,63.240335,2836.576690,2483.70
3,1,2024-06-10 00:00:00,48.153913,2486.759690,2421.99
4,1,2024-06-18 00:00:00,20.112090,815.611812,2230.23
...,...,...,...,...,...
5089,585,2024-08-29 00:00:00,5.704046,474.008068,27.02
5090,585,2024-09-04 00:00:00,NaN,NaN,719.45
5091,585,2024-09-12 00:00:00,NaN,NaN,115.90
5092,(en blanco),2024-09-12 00:00:00,NaN,NaN,22.92


In [36]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador
        
    #print(df_param)
    #print(suma_acumulador)
    df_param['ENTREGAS'] = suma_acumulador
    
    print(df_param)
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [37]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [38]:
list_df = []

In [39]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

    COD_COS      FECHA        AREA     ESTIMADO  ENTREGADO  ENTREGAS
0         1 2024-05-16    4.127295   221.918352       0.00      0.00
1         1 2024-05-31   45.972261  2338.489409    2876.77   2876.77
2         1 2024-06-05   63.240335  2836.576690    2483.70   2483.70
3         1 2024-06-10   48.153913  2486.759690    2421.99   2421.99
4         1 2024-06-18   20.112090   815.611812    2230.23   2230.23
5         1 2024-06-20   29.470226  2021.733284       0.00      0.00
6         1 2024-07-20   10.552630   619.266573    1136.33   1136.33
7         1 2024-07-25   33.115243  1651.084930    1916.31   1916.31
8         1 2024-08-11  111.015957  7411.451546    6811.41   6811.41
9         1 2024-08-15   21.668291  1388.784784    1112.92   1112.92
10        1 2024-08-19   22.008380  1162.986991     910.26    910.26
11        1 2024-08-29   84.790815  3131.051326    3644.64   3644.64
12        1 2024-09-04   44.708430  1397.414072    1577.55   1577.55
13        1 2024-09-12    8.662223

C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

     COD_COS      FECHA       AREA    ESTIMADO  ENTREGADO  ENTREGAS
480       54 2024-06-05   4.492558  189.755111       0.00      0.00
481       54 2024-06-10   1.840635   39.976601       0.00      0.00
482       54 2024-07-17   1.943651   99.431750       0.00      0.00
483       54 2024-07-20   4.055404  110.926377       0.00      0.00
484       54 2024-07-25   1.563505  104.528656       0.00      0.00
485       54 2024-08-11   2.762104  185.142720       0.00      0.00
486       54 2024-08-29   5.174992  308.014913     392.34    392.34
487       54 2024-09-04  18.512182  931.589886    1096.47   1096.47
488       54 2024-09-12   1.227518   52.729879       0.00      0.00
489       54 2024-09-18   3.307218  118.771668       0.00      0.00
     COD_COS      FECHA       AREA     ESTIMADO  ENTREGADO  ENTREGAS
490       55 2024-05-31   6.590077   388.805938     577.36    577.36
491       55 2024-06-05   0.000000     0.000000      87.73       NaN
492       55 2024-06-10   0.000000     0.0000

C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

...

C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

      COD_COS      FECHA  AREA  ESTIMADO  ENTREGADO  ENTREGAS
4237      443 2024-07-25   0.0       0.0      76.95       NaN
4238      443 2024-08-11   0.0       0.0      45.62    122.57
      COD_COS      FECHA       AREA     ESTIMADO  ENTREGADO  ENTREGAS
4239      444 2024-05-31   0.000000     0.000000    1338.07       NaN
4240      444 2024-06-18   0.000000     0.000000     264.24       NaN
4241      444 2024-06-20   0.000000     0.000000      65.74       NaN
4242      444 2024-07-17  68.980170  3586.264847    1729.34   3397.39
4243      444 2024-07-20  17.877142   753.384021     520.74    520.74
4244      444 2024-07-25   0.000000     0.000000     741.88       NaN
4245      444 2024-08-11   0.000000     0.000000     886.46       NaN
4246      444 2024-08-19   0.000000     0.000000     390.44       NaN
4247      444 2024-08-29   0.000000     0.000000     332.94   2351.72
      COD_COS      FECHA       AREA     ESTIMADO  ENTREGADO  ENTREGAS
4248      445 2024-05-31   1.121202    59.65

C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

      COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
4655      525 2024-08-11  0.000000    0.000000     131.38       NaN
4656      525 2024-08-15  5.107175  414.868173       0.00    131.38
      COD_COS      FECHA       AREA     ESTIMADO  ENTREGADO  ENTREGAS
4657      526 2024-07-17  52.138057  2224.694218    2201.22   2201.22
      COD_COS      FECHA       AREA    ESTIMADO  ENTREGADO  ENTREGAS
4658      527 2024-08-11   0.000000    0.000000     242.68       NaN
4659      527 2024-08-15  10.203191  504.357683     396.06    638.74
4660      527 2024-08-19   4.634758  205.032615     147.10    147.10
4661      527 2024-08-29   0.659589    5.375226       0.00      0.00
      COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
4662      528 2024-05-16  0.417742   24.944326       0.00      0.00
4663      528 2024-06-05  2.237382  135.966753       0.00      0.00
4664      528 2024-06-10  2.048633  108.925689      79.79     79.79
4665      528 2024-07-25  5.827705  220

C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_29828\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

In [40]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [41]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2024-05-16 00:00:00,4.127295,221.918352,0.00
1,1,2024-05-31 00:00:00,45.972261,2338.489409,2876.77
2,1,2024-06-05 00:00:00,63.240335,2836.576690,2483.70
3,1,2024-06-10 00:00:00,48.153913,2486.759690,2421.99
4,1,2024-06-18 00:00:00,20.112090,815.611812,2230.23
...,...,...,...,...,...
3670,585,2024-07-25 00:00:00,4.476407,274.635763,303.69
3671,585,2024-08-11 00:00:00,23.341766,1212.828644,233.24
3672,585,2024-08-29 00:00:00,5.704046,474.008068,258.69
3673,585,2024-09-12 00:00:00,0.000000,0.000000,835.35


In [42]:
df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')
#df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')